In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import sys
sys.path.insert(1, "../src/features")
import data_cleaning

# allows all columns to be displayed
pd.set_option('display.max_columns', None)

In [2]:
OFFER_DATA_DIR = "../data/offer_acceptance_offers.csv"
ORDER_DATA_DIR = "../data/offer_acceptance_orders.csv"

offers = pd.read_csv(OFFER_DATA_DIR, low_memory=False)[["CARRIER_ID", "REFERENCE_NUMBER", "CREATED_ON_HQ", "RATE_USD", "OFFER_TYPE", "LOAD_DELIVERED_FROM_OFFER"]]
orders = pd.read_csv(ORDER_DATA_DIR, low_memory=False)[["REFERENCE_NUMBER", "ORDER_DATETIME_PST", "PICKUP_DEADLINE_PST", "ORIGIN_3DIGIT_ZIP", "DESTINATION_3DIGIT_ZIP", "APPROXIMATE_DRIVING_ROUTE_MILEAGE", "PALLETIZED_LINEAR_FEET"]]

offers = data_cleaning.change_to_date(offers, ["CREATED_ON_HQ"])
orders = data_cleaning.change_to_date(orders, ["ORDER_DATETIME_PST", "PICKUP_DEADLINE_PST"])

orders = data_cleaning.parse_zipcode(orders)

orders = data_cleaning.parse_datetime(orders)

offers = data_cleaning.flatten_ref_num(offers)
orders = data_cleaning.flatten_ref_num(orders)

merged = data_cleaning.join_offers_orders(offers, orders, how="inner")

merged = data_cleaning.get_remaining_time(merged)

merged = data_cleaning.during_business_hours(merged)

pooled = data_cleaning.get_prorated_rate(merged)

merged = data_cleaning.impute_mileage(merged)

merged = data_cleaning.get_business_hours(merged)

In [4]:
pooled

,CARRIER_ID,REFERENCE_NUMBER,CREATED_ON_HQ,RATE_USD,LOAD_DELIVERED_FROM_OFFER,ORDER_DATETIME_PST,PICKUP_DEADLINE_PST,ORIGIN_3DIGIT_ZIP,DESTINATION_3DIGIT_ZIP,APPROXIMATE_DRIVING_ROUTE_MILEAGE,PALLETIZED_LINEAR_FEET,ORIGIN_CITY,DESTINATION_CITY,ORDER_DAY,ORDER_MONTH,ORDER_HOUR,PICKUP_DAY,PICKUP_MONTH,PICKUP_HOUR,REMAINIG_TIME,BUSINESS_HOURS,PRORATED_RATE_USD
0,16badfc6202cb3f8889e0f2779b19218af4cbb736e56ac...,ce3548db155049cc1ccce2da041cec607942e4f779fc2d...,2021-11-03 09:05:43,11610.0,False,2021-11-02 12:56:49,2021-11-03 16:00:00,945,601,2131.0,4,Oakland CA,Chicago IL,1,11,12,2,11,16,24857.0,True,967.50
1,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,ce3548db155049cc1ccce2da041cec607942e4f779fc2d...,2021-11-03 08:54:13,9546.0,False,2021-11-02 12:56:49,2021-11-03 16:00:00,945,601,2131.0,4,Oakland CA,Chicago IL,1,11,12,2,11,16,25547.0,True,795.50
2,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,ce3548db155049cc1ccce2da041cec607942e4f779fc2d...,2021-11-03 08:57:27,9159.0,True,2021-11-02 12:56:49,2021-11-03 16:00:00,945,601,2131.0,4,Oakland CA,Chicago IL,1,11,12,2,11,16,25353.0,True,763.25
3,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,ce3548db155049cc1ccce2da041cec607942e4f779fc2d...,2021-11-03 08:57:28,9288.0,False,2021-11-02 12:56:49,2021-11-03 16:00:00,945,601,2131.0,4,Oakland CA,Chicago IL,1,11,12,2,11,16,25352.0,True,774.00
4,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,ce3548db155049cc1ccce2da041cec607942e4f779fc2d...,2021-11-03 08:57:40,9159.0,False,2021-11-02 12:56:49,2021-11-03 16:00:00,945,601,2131.0,4,Oakland CA,Chicago IL,1,11,12,2,11,16,25340.0,True,763.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237237,ad7c2b601d56bacae7e4dcd1d4e0598f151162a0513749...,a2d8690d86bf84d3ac272158cce83bb4f73c53933cf585...,2020-06-11 15:17:33,5031.0,False,2020-06-11 09:23:33,2020-06-11 15:30:00,925,458,2259.0,16,Riverside CA,Lima OH,3,6,9,3,6,15,747.0,True,2515.50
237238,a73822c40544527c65f2ce9704d36c48b9eb2d9404b187...,2292310ba6562814a95f139a38be2064f51adb2b8eb9c5...,2020-06-11 14:29:51,5147.1,False,2020-06-11 10:43:15,2020-06-11 15:30:00,925,633,1834.0,16,Riverside CA,St. Charles MO,3,6,10,3,6,15,3609.0,True,2573.55
237239,c83476e39f7dbae15db106e746e0f125ac07ae22416c8f...,2292310ba6562814a95f139a38be2064f51adb2b8eb9c5...,2020-06-11 14:15:55,9675.0,False,2020-06-11 10:43:15,2020-06-11 15:30:00,925,633,1834.0,16,Riverside CA,St. Charles MO,3,6,10,3,6,15,4445.0,True,4837.50
237240,eb282120e33fdf33085803b3f8d0409837b07a9994d981...,2292310ba6562814a95f139a38be2064f51adb2b8eb9c5...,2020-06-11 13:20:28,5418.0,False,2020-06-11 10:43:15,2020-06-11 15:30:00,925,633,1834.0,16,Riverside CA,St. Charles MO,3,6,10,3,6,15,7772.0,True,2709.00


In [39]:
df_full = merged
max_categories=30

"""
Creates pipeline for regression model to predict RATE of an item having "quote" label.

Args:
    df_full (DataFrame): dataframe to train model on
    max_categories (int, optional): maximum number of categorical values to be onehot encoded. Defaults to 30.

Returns:
    Pipeline: pipeline of feature transformation and model
    df_X_test: DataFrame of test set features
    df_y_test: DataFrame of test set labels
"""
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

def grid_search(pl, df_X_train, df_y_train):
  """
  Grid search for regression model to find out the best parameter.

  Args:
      pl (Pipeline): Pipeline that builds regression model
      df_X_train (DataFrame): train dataframe of X value.
      df_y_train (DataFrame): train dataframe of y value.

  Returns:
      grid_pipeline.best_params_ : The best parameter from  range 0.01 to 10
  """
  parameters = {
      "regressor__alpha":[0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
  }

  # initialize
  grid_pipeline = GridSearchCV(pl,parameters, cv = 8)

  grid_pipeline.fit(df_X_train,df_y_train)

  return grid_pipeline.best_params_

# check df_full is a DataFrame
if not isinstance(df_full, pd.DataFrame): AssertionError("Parameter must be Pandas DataFrame")

# checks if dataframe has required columns
req_cols = ['RATE_USD', 'APPROXIMATE_DRIVING_ROUTE_MILEAGE', 'PALLETIZED_LINEAR_FEET',
      'ORIGIN_CITY', 'DESTINATION_CITY', 'ORDER_DAY', 'ORDER_MONTH',
      'ORDER_HOUR', 'PICKUP_DAY', 'PICKUP_MONTH', 'PICKUP_HOUR',
      'REMAINIG_TIME', 'BUSINESS_HOURS', 'BUSINESS_HOURS_ORDER_PICKUP', "OFFER_TYPE"]

if not set(req_cols).issubset(set(df_full.columns)): AssertionError("DataFrame does not contain required columns")

# filter for delivered offers
df_full = df_full[df_full["LOAD_DELIVERED_FROM_OFFER"]].reset_index(drop=True)

# select required columns only 
df_full = df_full[req_cols]

# only quote
df_full = df_full[df_full["OFFER_TYPE"] == "quote"]
df_full = df_full.drop(["OFFER_TYPE"], axis=1)

# split features and labels
df_X = df_full.drop(["RATE_USD"], axis=1)
df_y = df_full["RATE_USD"]

# split train test
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=42)

# create numerical value transformer
num_feat = ["APPROXIMATE_DRIVING_ROUTE_MILEAGE", "PALLETIZED_LINEAR_FEET", "REMAINIG_TIME", 'BUSINESS_HOURS_ORDER_PICKUP']
num_transformer = Pipeline(steps=[
    ('scaler', MaxAbsScaler())
])

cat_feat = ['ORIGIN_CITY', 'DESTINATION_CITY', 'BUSINESS_HOURS', 'ORDER_DAY', 'ORDER_MONTH', 'ORDER_HOUR', 'PICKUP_DAY', 'PICKUP_MONTH', 'PICKUP_HOUR']
cat_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(max_categories = max_categories, handle_unknown = 'ignore'))     # output from Ordinal becomes input to OneHot
])

# combine numerical and categorical transformers
preproc = ColumnTransformer(
  transformers=[
      ("numerical", num_transformer, num_feat),
      ("categorization", cat_transformer, cat_feat)
  ])

In [44]:
np.arange(0.01, 10, 0.01)

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  , 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1 ,
       1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2 , 1.21,
       1.22, 1.23, 1.24, 1.25, 1.26, 1.27, 1.28, 1.29, 1.3 , 1.31, 1.32,
       1.33, 1.34, 1.35, 1.36, 1.37, 1.38, 1.39, 1.4 , 1.41, 1.42, 1.43,
       1.44, 1.45, 1.46, 1.47, 1.48, 1.49, 1.5 , 1.

In [45]:
pl = Pipeline(steps=[('preprocessor', preproc), ('regressor', Ridge(alpha = 1))])

parameters = {
    "regressor__alpha":np.arange(0.01, 5, 0.01)
}

# initialize
grid_pipeline = GridSearchCV(pl,parameters, cv = 8)

grid_pipeline.fit(df_X_train,df_y_train)

grid_pipeline.best_params_

{'regressor__alpha': 2.3299999999999996}

In [ ]:
pl = Pipeline(steps=[('preprocessor', preproc), ('regressor', Lasso(alpha = 1))])

parameters = {
    "regressor__alpha":np.arange(0.01, 5, 0.01)
}

# initialize
grid_pipeline = GridSearchCV(pl,parameters, cv = 8)

grid_pipeline.fit(df_X_train,df_y_train)

grid_pipeline.best_params_

In [54]:
pl = Pipeline(steps=[('preprocessor', preproc), ('regressor', Ridge(alpha = 0.01))])

# train model
pl.fit(df_X_train, df_y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   MaxAbsScaler())]),
                                                  ['APPROXIMATE_DRIVING_ROUTE_MILEAGE',
                                                   'PALLETIZED_LINEAR_FEET',
                                                   'REMAINIG_TIME',
                                                   'BUSINESS_HOURS_ORDER_PICKUP']),
                                                 ('categorization',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 max_categories=30))]),
                                                  ['ORIGIN_CITY',
                                                   'DESTINATION_CITY',
                                                   'BUSINESS_HOURS',
                                                   'ORDER_DAY', 'ORDER_MONTH',
                                                   'ORDER_HOUR', 'PICKUP_DAY',
                                                   'PICKUP_MONTH',
                                                   'PICKUP_HOUR'])])),
                ('regressor', Ridge(alpha=0.01))])

In [55]:
pl.fit(df_X_test, df_y_test)

_preds = pl.predict(df_X_test)

mse = mean_squared_error(df_y_test, _preds)
print("MSE of %s is %s"%("Ridge", mse))

MSE of Ridge is 1257331.3786684826


In [ ]:
1259404.2505492966

In [3]:
import sys
from sklearn.metrics import mean_squared_error
sys.path.insert(1, "../src/models")

import quote_regression_model

pl, (df_X_test, df_y_test) = quote_regression_model.generate_quote_regression_pipeline(merged)

pl.fit(df_X_test, df_y_test)

_preds = pl.predict(df_X_test)

mse = mean_squared_error(df_y_test, _preds)
print("MSE of %s is %s"%("Ridge", mse))

MSE of Ridge is 1258285.425800086


In [3]:
 # filter for delivered offers
merged = merged[merged["LOAD_DELIVERED_FROM_OFFER"]].reset_index(drop=True)

 # filter for quote offers
merged = merged[merged["OFFER_TYPE"] == "quote"]

# drop unnecessary columns
merged = merged.drop(["CARRIER_ID", "REFERENCE_NUMBER", "LOAD_DELIVERED_FROM_OFFER", "CREATED_ON_HQ", "ORIGIN_3DIGIT_ZIP", "DESTINATION_3DIGIT_ZIP"], axis=1)

In [4]:
merged

,RATE_USD,OFFER_TYPE,ORDER_DATETIME_PST,PICKUP_DEADLINE_PST,APPROXIMATE_DRIVING_ROUTE_MILEAGE,PALLETIZED_LINEAR_FEET,ORIGIN_CITY,DESTINATION_CITY,ORDER_DAY,ORDER_MONTH,ORDER_HOUR,PICKUP_DAY,PICKUP_MONTH,PICKUP_HOUR,REMAINIG_TIME,BUSINESS_HOURS,BUSINESS_HOURS_ORDER_PICKUP
2,1096.5000,quote,2022-07-21 06:17:07,2022-07-28 13:00:00,318.0,14,Rancho Cucamonga CA,Glendale AZ,3,7,6,3,7,13,470721.0,False,55.000000
3,1806.0000,quote,2022-07-26 10:22:11,2022-07-28 14:00:00,368.0,31,Whittier CA,Phoenix AZ,1,7,10,3,7,14,89198.0,True,23.630278
7,3870.0000,quote,2022-09-08 13:19:35,2022-09-09 13:00:00,2659.0,15,Whittier CA,West Palm Beach FL,3,9,13,4,9,13,15004.0,True,9.673611
18,4902.0000,quote,2022-03-08 09:01:18,2022-03-11 16:00:00,2186.0,32,San Bernardino CA,Gainesville GA,1,3,9,4,3,16,83617.0,True,36.978333
21,1935.0000,quote,2022-03-04 10:58:02,2022-03-08 11:00:00,966.0,21,Dallas TX,Chicago IL,4,3,10,1,3,11,18432.0,True,20.032778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161676,967.5000,quote,2022-06-28 11:20:10,2022-06-29 14:00:00,408.0,52,Oakland CA,Whittier CA,1,6,11,2,6,14,95950.0,True,12.663889
161677,1032.0000,quote,2020-10-22 05:30:25,2020-10-23 11:30:00,413.0,52,Atlanta GA,Jackson MS,3,10,5,4,10,11,107099.0,True,13.500000
161678,1290.0000,quote,2021-08-17 09:25:59,2021-08-18 13:00:00,397.0,22,Dallas TX,Hutchinson KS,1,8,9,2,8,13,90673.0,True,13.566944
161679,6450.0000,quote,2021-06-29 05:18:36,2021-06-30 16:00:00,1080.0,29,San Diego CA,Longmont CO,1,6,5,2,6,16,28273.0,True,18.000000


In [7]:
merged.columns

Index(['RATE_USD', 'OFFER_TYPE', 'ORDER_DATETIME_PST', 'PICKUP_DEADLINE_PST',
       'APPROXIMATE_DRIVING_ROUTE_MILEAGE', 'PALLETIZED_LINEAR_FEET',
       'ORIGIN_CITY', 'DESTINATION_CITY', 'ORDER_DAY', 'ORDER_MONTH',
       'ORDER_HOUR', 'PICKUP_DAY', 'PICKUP_MONTH', 'PICKUP_HOUR',
       'REMAINIG_TIME', 'BUSINESS_HOURS', 'BUSINESS_HOURS_ORDER_PICKUP'],
      dtype='object')

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [12]:
def generate_probability_pipeline(df):
  
  MAX_CATEGORIES = 30

  df = df.drop(["ORDER_DATETIME_PST", "PICKUP_DEADLINE_PST", "OFFER_TYPE"], axis=1)

  df_X = df.drop("RATE_USD", axis=1)
  df_y = df["RATE_USD"]

  df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=42)

  if not [col in df.cols for col in df_X].mean() == 1:
    AssertionError("DataFrame does not contain required columns")

  num_feat = ["APPROXIMATE_DRIVING_ROUTE_MILEAGE", "PALLETIZED_LINEAR_FEET", "REMAINIG_TIME", 'BUSINESS_HOURS_ORDER_PICKUP']
  num_transformer = Pipeline(steps=[
      ('scaler', MaxAbsScaler())
  ])

  cat_feat = ['ORIGIN_CITY', 'DESTINATION_CITY', 'BUSINESS_HOURS', 'ORDER_DAY', 'ORDER_MONTH', 'ORDER_HOUR', 'PICKUP_DAY', 'PICKUP_MONTH', 'PICKUP_HOUR']
  cat_transformer = Pipeline(steps=[
      ('onehot', OneHotEncoder(max_categories = 30, handle_unknown = 'ignore'))     # output from Ordinal becomes input to OneHot
  ])

  # preprocessing pipeline (put them together)
  preproc = ColumnTransformer(
      transformers=[
          ("numerical", num_transformer, num_feat),
          ("categorization", cat_transformer, cat_feat)
      ])

  pipeline = ...

  return pipeline

In [13]:
MAX_CATEGORIES = 30

df = merged.drop(["ORDER_DATETIME_PST", "PICKUP_DEADLINE_PST", "OFFER_TYPE"], axis=1)

df_X = df.drop("RATE_USD", axis=1)
df_y = df["RATE_USD"]

df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=42)

num_feat = ["APPROXIMATE_DRIVING_ROUTE_MILEAGE", "PALLETIZED_LINEAR_FEET", "REMAINIG_TIME", 'BUSINESS_HOURS_ORDER_PICKUP']
num_transformer = Pipeline(steps=[
    ('scaler', MaxAbsScaler())
])

cat_feat = ['ORIGIN_CITY', 'DESTINATION_CITY', 'BUSINESS_HOURS', 'ORDER_DAY', 'ORDER_MONTH', 'ORDER_HOUR', 'PICKUP_DAY', 'PICKUP_MONTH', 'PICKUP_HOUR']
cat_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(max_categories = 30, handle_unknown = 'ignore'))     # output from Ordinal becomes input to OneHot
])

# preprocessing pipeline (put them together)
preproc = ColumnTransformer(
    transformers=[
        ("numerical", num_transformer, num_feat),
        ("categorization", cat_transformer, cat_feat)
    ])

In [14]:
regressors = []

lasso_model = Lasso()
regressors.append(lasso_model)

gidge_model = Ridge()
regressors.append(gidge_model)

for reg in regressors:
    pl = Pipeline(steps=[('preprocessor', preproc), (str(reg), reg)])
    
    pl.fit(df_X_train, df_y_train)
    y_preds = pl.predict(df_X_test)

    mse = mean_squared_error(df_y_test, y_preds)
    print("MSE of %s is %s"%(reg, mse))

MSE of Lasso() is 1264086.2147975056
MSE of Ridge() is 1258982.4854549037


In [38]:
[i for i in np.arange(0.1, 10, 0.1)]

[0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 1.1,
 1.2000000000000002,
 1.3000000000000003,
 1.4000000000000001,
 1.5000000000000002,
 1.6,
 1.7000000000000002,
 1.8000000000000003,
 1.9000000000000001,
 2.0,
 2.1,
 2.2,
 2.3000000000000003,
 2.4000000000000004,
 2.5000000000000004,
 2.6,
 2.7,
 2.8000000000000003,
 2.9000000000000004,
 3.0000000000000004,
 3.1,
 3.2,
 3.3000000000000003,
 3.4000000000000004,
 3.5000000000000004,
 3.6,
 3.7,
 3.8000000000000003,
 3.9000000000000004,
 4.0,
 4.1,
 4.2,
 4.3,
 4.3999999999999995,
 4.5,
 4.6,
 4.7,
 4.8,
 4.9,
 5.0,
 5.1,
 5.2,
 5.3,
 5.4,
 5.5,
 5.6,
 5.7,
 5.8,
 5.9,
 6.0,
 6.1,
 6.2,
 6.3,
 6.4,
 6.5,
 6.6,
 6.7,
 6.8,
 6.9,
 7.0,
 7.1,
 7.2,
 7.3,
 7.4,
 7.5,
 7.6,
 7.7,
 7.8,
 7.9,
 8.0,
 8.1,
 8.2,
 8.3,
 8.4,
 8.5,
 8.6,
 8.7,
 8.8,
 8.9,
 9.0,
 9.1,
 9.2,
 9.3,
 9.4,
 9.5,
 9.6,
 9.700000000000001,
 9.8,
 9.9]

In [22]:
pl = Pipeline(steps=[('preprocessor', preproc), ('regressor', Ridge())])

parameters = {
    "regressor__alpha":[i / 10 for i in range(1, 100)]
}


# initialize
grid_pipeline = GridSearchCV(pl,parameters, cv = 8)


grid_pipeline.fit(df_X_train,df_y_train)

grid_pipeline.best_params_

{'regressor__alpha': 2.33}

In [25]:
grid_pipeline.best_params_['regressor__alpha']

2.33

In [24]:
pl = Pipeline(steps=[('preprocessor', preproc), ('regressor', Ridge(alpha = 2.3))])

pl.fit(df_X_train, df_y_train)
y_preds = pl.predict(df_X_test)

mse = mean_squared_error(df_y_test, y_preds)
mse

1258774.8141576087

In [37]:
pl, (df_X_test, df_y_test) = generate_probability_pipeline(merged)

In [38]:
pl.fit(df_X_train, df_y_train)

_preds = pl.predict(df_X_test)

mse = mean_squared_error(df_y_test, y_preds)
print("MSE of %s is %s"%(reg, mse))

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   MaxAbsScaler())]),
                                                  ['APPROXIMATE_DRIVING_ROUTE_MILEAGE',
                                                   'PALLETIZED_LINEAR_FEET',
                                                   'REMAINIG_TIME',
                                                   'BUSINESS_HOURS_ORDER_PICKUP']),
                                                 ('categorization',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 max_categories=30))]),
                                                  ['ORIGIN_CITY',
                                                   'DESTINATION_CITY',
                                                   'BUSINESS_HOURS',
                                                   'ORDER_DAY', 'ORDER_MONTH',
                                                   'ORDER_HOUR', 'PICKUP_DAY',
                                                   'PICKUP_MONTH',
                                                   'PICKUP_HOUR'])])),
                ('regressor', Ridge(alpha=2.3))])

In [39]:
_preds = pl.predict(df_X_test)

mse = mean_squared_error(df_y_test, y_preds)
print("MSE of %s is %s"%(reg, mse))

MSE of Ridge() is 1258774.8141576087
